In [2]:
!pip install langchain langchain-openai langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 74.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.8/289.8 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.1/374.1 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/4

In [6]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 99.6 MB/s eta 0:00:00


In [19]:
os.environ["OPENAI_API_KEY"] = ''

In [21]:
!pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.8 MB/s eta 0:00:00


In [ ]:
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
import os
import fitz

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
pdf_file_path = '/content/gdrive/MyDrive/RAG_paper.pdf'

In [ ]:
#get text from pdf file
def get_text_from_pdf(pdf_file_path):
  doc = fitz.open(pdf_file_path)
  text=""
  for page in doc:
      text = text+ page.get_text()
  return text

In [ ]:
#split text into chunks
def get_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    text_chunks = text_splitter.split_text(text)
    return text_chunks

In [ ]:
#embeddings for chunks
def get_embeddings(chunks):
    embeddings = OpenAIEmbeddings()
    vector_storage = FAISS.from_texts(texts=chunks, embedding=embeddings)
    return vector_storage


In [ ]:
#setting up the conversation
def start_conversation(vector_embeddings):
    llm = ChatOpenAI(temperature=0.9, model_name="gpt-4o")
    memory = ConversationBufferMemory(
        memory_key='chat_history',
        return_messages=True
    )
    conversation = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vector_embeddings.as_retriever(),
        memory=memory
    )
    return conversation

In [ ]:
def main():
    text = get_text_from_pdf(pdf_file_path)
    chunks = get_chunks(text)
    vector_embeddings = get_embeddings(chunks)
    conversation = start_conversation(vector_embeddings)
    return conversation
conversation = main()

In [29]:
#test the conversation
query = "What is RAG?"
response = conversation.run(query)
print(response)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
RAG (Retrieval-Augmented Generation) is a model that combines the flexibility of generative (closed-book) approaches with the performance of retrieval-based (open-book) approaches. It generates answers and other text by retrieving relevant documents and using this information to inform its generation process. This model does not require an extractive reader or specialized pre-training, which distinguishes it from other models like REALM and T5+SSM.

Key features of RAG include:
- Performing state-of-the-art on open-domain question answering tasks.
- Initializing its retriever using Dense Passage Retrieval (DPR) models, which are supervised with retrieval tasks on Natural Questions and TriviaQA.
- Generating more factually correct and diverse text compared to models like BART.
- Combining content from several documents to generate responses, making it partic